# lab 1.2 Part 2

In [5]:
import arcpy
import psycopg2
from psycopg2 import sql
from psycopg2.extras import RealDictCursor

ModuleNotFoundError: No module named 'psycopg2'

In [3]:
pointArray = arcpy.Array([arcpy.Point(12,16),
                          arcpy.Point(51,2),
                          arcpy.Point(23,32),
                          arcpy.Point(7,4)])

polygon = arcpy.Polygon(pointArray)
polygon_wkt = polygon.WKT

In [4]:
connection = psycopg2.connect(host = '34.27.240.213',
                             database = 'lab0',
                             user = 'postgres',
                             password = 'Homework13'
                             )

cursor = connection.cursor()

# Define the SQL command to insert the polygon into the database
sql = "INSERT INTO geom_table (columnLocation) VALUES (ST_GeomFromText('{}', 4326))".format(polygon_wkt)

# Execute the SQL command
cursor.execute(sql) # do some sql commands

# Commit the changes to the database
connection.commit()

# Close the cursor
cursor.close()

# Close the connection
connection.close()

In [13]:
import requests

url = "http://34.83.204.105:5000/get_polygon"
response = requests.get(url)

if response.status_code == 200:
    geojson = response.json()
    with open("C:/Users/swimb/Desktop/geojson", "w") as f:
        f.write(str(geojson))
        print("GeoJSON file saved to C:/Users/swimb/Desktop/geojson")
else:
    print("Failed to retrieve geojson data from the URL.")


PermissionError: [Errno 13] Permission denied: 'C:/Users/swimb/Documents/UMN/spring2023/Arc2'

In [ ]:
import json

# First JSON
json_string_1 = '{"geom_table":"(,01060000000200000001030000000100000004000000000000001000284000000000080030400000000020001C4000000000200010400000000008152B4000000000D0E42E4000000000100028400000000008003040010300000001000000040000000000000008152B4000000000D0E42E4000000000048049400000000040000040000000000800374000000000040040400000000008152B4000000000D0E42E40)","geometry":{"coordinates":[[[[12.00012207,16.00012207],[7.00012207,4.00012207],[13.54107666,15.446899414],[12.00012207,16.00012207]]],[[[13.54107666,15.446899414],[51.00012207,2.00012207],[23.00012207,32.00012207],[13.54107666,15.446899414]]]],"type":"MultiPolygon"}}'

# Load the JSON string as a Python object
json_obj_1 = json.loads(json_string_1)

# Convert the first JSON object to the second JSON object
json_obj_2 = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": json_obj_1['geometry'],
            "properties": {
                "geom_table": json_obj_1['geom_table']
            }
        }
    ]
}

# Convert the Python object to a JSON string
json_string_2 = json.dumps(json_obj_2)

# Print the resulting JSON string
print(json_string_2)

# Save the dictionary as a JSON file
with open('C:/Users/swimb/Desktop/geojson/esri_json.json', 'w') as json_file:
    json.dump(json_obj_2, json_file)

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Connect to your ArcGIS Online organization
gis = GIS("https://www.arcgis.com", "swimbott13", "....")

# Upload the GeoJSON file as a new item
item = gis.content.add({'type': 'Shapefile'}, 'C:/Users/swimb/Desktop/geojson/final_esri_json.shp')

# Publish the new item as a feature layer
feature_layer_item = item.publish()

# Get the feature layer URL and print it
feature_layer_url = feature_layer_item.url
print(feature_layer_url)